<a href="https://colab.research.google.com/github/singhbhupender1/ML-notebooks/blob/master/How_to_Manually_Scale_Image_Pixel_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Manually Scale Image Pixel Data for Deep Learning

**Black and white images are single matrix of pixels, whereas color images have a separate array of pixel values for each color channel, such as red, green, and blue.**

**Pixel values are often unsigned integers in the range between 0 and 255. Although these pixel values can be presented directly to neural network models in their raw format, this can result in challenges during modeling, such as in the slower than expected training of the model.**

**Instead, there can be great benefit in preparing the image pixel values prior to modeling, such as simply scaling pixel values to the range 0-1 to centering and even standardizing the values.**

In [1]:
#load and show an image with Pillow
from PIL import Image
from urllib.request import urlopen
#load the image
url  = 'https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/01/The-Sydney-Harbor-Bridge-Photograph-Loaded-From-File.png'
image = Image.open(urlopen(url))
#Summerize some details about the Image
print(image.format)
print(image.mode)
print(image.size)
#show the image
image.show()

PNG
RGBA
(1280, 856)


##Normalize Pixel Values

**For most image data, the pixel values are integers with values between 0 and 255.**

**Neural networks process inputs using small weight values, and inputs with large integer values can disrupt or slow down the learning process. As such it is good practice to normalize the pixel values so that each pixel value has a value between 0 and 1.**

**It is valid for images to have pixel values in the range 0-1 and images can be viewed normally.**

**This can be achieved by dividing all pixels values by the largest pixel value; that is 255. This is performed across all channels, regardless of the actual range of pixel values that are present in the image.**

**The example below loads the image and converts it into a NumPy array. The data type of the array is reported and the minimum and maximum pixels values across all three channels are then printed. Next, the array is converted to the float data type before the pixel values are normalized and the new range of pixel values is reported.**

In [2]:
#example of pixel normalization
from numpy import asarray
from PIL import Image
from urllib.request import urlopen
#load image
url = 'https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/01/The-Sydney-Harbor-Bridge-Photograph-Loaded-From-File.png'
img = Image.open(urlopen(url))
pixels = asarray(img)
#confirm pixel range is 0-255
print('Data type : %s' % pixels.dtype)
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
#convert from intergers to floats
pixels = pixels.astype('float32')
#normalize to the range 0-1
pixels /= 255.0
#confirm the normalization
print('Min: %.3f, Max:%.3f' % (pixels.min(), pixels.max()))

Data type : uint8
Min: 0.000, Max: 255.000
Min: 0.000, Max:1.000


**Normalization is a good default data preparation that can be performed if you are in doubt as to the type of data preparation to perform.**

**It can be performed per image and does not require the calculation of statistics across the training dataset, as the range of pixel values is a domain standard.**

##Center Pixel Values

**A popular data preparation technique for image data is to subtract the mean value from the pixel values.**

**This approach is called centering, as the distribution of the pixel values is centered on the value of zero.**

**Centering can be performed before or after normalization. Centering the pixels then normalizing will mean that the pixel values will be centered close to 0.5 and be in the range 0-1. Centering after normalization will mean that the pixels will have positive and negative values, in which case images will not display correctly (e.g. pixels are expected to have value in the range 0-255 or 0-1). Centering after normalization might be preferred, although it might be worth testing both approaches.**

**Centering requires that a mean pixel value be calculated prior to subtracting it from the pixel values. There are multiple ways that the mean can be calculated; for example:**

>**Per image.**

>**Per mini-batch of images (under stochastic gradient descent).**

>**Per training dataset.**

**The mean can be calculated for all pixels in the image, referred to as a global centering, or it can be calculated for each channel in the case of color images, referred to as local centering.**

**Global Centering: Calculating and subtracting the mean pixel value across color channels.**

**Local Centering: Calculating and subtracting the mean pixel value per color channel.**

**Per-image global centering is common because it is trivial to implement. Also common is per mini-batch global or local centering for the same reason: it is fast and easy to implement.**

**In some cases, per-channel means are pre-calculated across an entire training dataset. In this case, the image means must be stored and used both during training and any inference with the trained models in the future. For example, the per-channel pixel means calculated for the ImageNet training dataset are as follows:**

**ImageNet Training Dataset Means: [0.485, 0.456, 0.406]**

**For models trained on images centered using these means that may be used for transfer learning on new tasks, it can be beneficial or even required to normalize images for the new task using the same means.**

##Global Centering

In [3]:
#example of global centering(subtract mean)
from numpy import asarray
from PIL import Image
from urllib.request import urlopen
#load image
url = 'https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/01/The-Sydney-Harbor-Bridge-Photograph-Loaded-From-File.png'
image = Image.open(urlopen(url))
pixels = asarray(image)
#convert from intergers to floats
pixels = pixels.astype('float32')
#calculate gloabal mean
mean = pixels.mean()
print('Mean:%.3f' % mean)
print('Min:%.3f, Max:%.3f' % (pixels.min(), pixels.max()))
#global centering of pixels
pixels = pixels - mean
#confirm it had the desired effect
mean = pixels.mean()
print('Mean: %.3f' % mean)
print('Min:%.3f, Max:%.3f' % (pixels.min(), pixels.max()))

Mean:184.501
Min:0.000, Max:255.000
Mean: -0.000
Min:-184.501, Max:70.499


##Local Centering

**NumPy allows us to specify the dimensions over which a statistic like the mean, min, and max are calculated via the “axis” argument. In this example, we set this to (0,1) for the width and height dimensions, which leaves the third dimension or channels. The result is one mean, min, or max for each of the three channel arrays.**

**when we calculate the mean that we specify the dtype as ‘float64‘; this is required as it will cause all sub-operations of the mean, such as the sum, to be performed with 64-bit precision. Without this, the sum will be performed at lower resolution and the resulting mean will be wrong given the accumulated errors in the loss of precision, in turn meaning the mean of the centered pixel values for each channel will not be zero (or a very small number close to zero)**

In [5]:
#examplr of per-channel centering(subtract mean)
from numpy import asarray
from PIL import Image
from urllib.request import urlopen
#load image
url = 'https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/01/The-Sydney-Harbor-Bridge-Photograph-Loaded-From-File.png'
image = Image.open(urlopen(url))
pixels = asarray(image)
#convert from integers to pixels
pixels = pixels.astype('float32')
#calculate per-channel menas and standard deviations
means = pixels.mean(axis=(0,1), dtype='float64')
print('Mean: %s' %means)
print('Mins:%s, Maxs:%s' % (pixels.min(axis=(0,1)), pixels.max(axis=(0,1))))
#per-channel centering of pixels
pixels -= means
#confirm it has the desired effect
means = pixels.mean(axis=(0,1), dtype='float64')
print('Means: %s' % means)
print('Min:%s, Max:%s' % (pixels.min(axis=(0,1)), pixels.max(axis=(0,1))))

Mean: [158.43480487 159.58662109 164.9829202  255.        ]
Mins:[  0.   0.   0. 255.], Maxs:[255. 255. 255. 255.]
Means: [-3.06365524e-07 -1.24562507e-06  4.88580506e-07  0.00000000e+00]
Min:[-158.4348  -159.58662 -164.98293    0.     ], Max:[96.56519  95.413376 90.01708   0.      ]


##Standardize Pixel Values

**The distribution of pixel values often follows a Normal or Gaussian distribution, e.g. bell shape.**

**This distribution may be present per image, per mini-batch of images, or across the training dataset and globally or per channel.**

**As such, there may be benefit in transforming the distribution of pixel values to be a standard Gaussian: that is both centering the pixel values on zero and normalizing the values by the standard deviation. The result is a standard Gaussian of pixel values with a mean of 0.0 and a standard deviation of 1.0.**

**As with centering, the operation can be performed per image, per mini-batch, and across the entire training dataset, and it can be performed globally across channels or locally per channel.**

**Standardization may be preferred to normalization and centering alone and it results in both zero-centered values of small input values, roughly in the range -3 to 3, depending on the specifics of the dataset.**

**For consistency of the input data, it may make more sense to standardize images per-channel using statistics calculated per mini-batch or across the training dataset, if possible.**

##Global Standardization

In [6]:
#example of global pixel standardization
from numpy import asarray
from PIL import Image
from urllib.request import urlopen
#load the image
url = 'https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/01/The-Sydney-Harbor-Bridge-Photograph-Loaded-From-File.png'
image = Image.open(urlopen(url))
pixels = asarray(image)
#convert from integers to floats
pixels = pixels.astype('float32')
#calculate global mean and standard deviation
mean, std = pixels.mean(), pixels.std()
print('Mean: %.3f, Standard deviation:%.3f' % (mean, std))
#global standardization of pixels
pixels = (pixels - mean)/ std
#confirm it had the desird effect
mean, std = pixels.mean(), pixels.std()
print('Means: %.3f, standard deviation:%.3f' % (mean, std))

Mean: 184.501, Standard deviation:73.418
Means: -0.000, standard deviation:1.000


##Positive Global Standardization

**There may be a desire to maintain the pixel values in the positive domain, perhaps so the images can be visualized or perhaps for the benefit of a chosen activation function in the model.**

**A popular way of achieving this is to clip the standardized pixel values to the range [-1, 1] and then rescale the values from [-1,1] to [0,1].**

In [9]:
#example of global pixels standardizaton shifted to positive domain
from numpy import asarray
from numpy import clip
from PIL import Image
from urllib.request import urlopen
#load image
url = 'https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/01/The-Sydney-Harbor-Bridge-Photograph-Loaded-From-File.png'
pixels = Image.open(urlopen(url))
pixels = asarray(pixels)
#convert from intefers to float
pixels = pixels.astype('float32')
#calcualte global mean and standard deviation
mean, std = pixels.mean(), pixels.std()
print('Mean:%.3f, standard deviation:%.3f' % (mean, std))
#global standardization of pixels
pixels = (pixels - mean) / std
#clip pixel values ot [-1, 1]
pixels = clip(pixels, -1.0, 1.0)
#shift from [-1, 1] to [0,1] with 0.5 mean
pixels = (pixels + 1.0) / 2.0
#confirm it has the desired effect
mean, std = pixels.mean(), pixels.std()
print('Mean:%.3f, standard deviation:%.3f' % (mean, std))
print('Min:%.3f, Max:%.3f' % (pixels.min(), pixels.max())) 

Mean:184.501, standard deviation:73.418
Mean:0.563, standard deviation:0.396
Min:0.000, Max:0.980


##Local Standardization

In [10]:
#example of per-channe; pixel standardization
from numpy import asarray
from PIL import Image
from urllib.request import urlopen
#load image
url = 'https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/01/The-Sydney-Harbor-Bridge-Photograph-Loaded-From-File.png'
image = Image.open(urlopen(url))
pixels = asarray(image)
#convert from integers to floats
pixels = pixels.astype('float32')
#calculate per-channel means and standard deviations
means = pixels.mean(axis=(0,1), dtype='float64')
stds = pixels.std(axis=(0,1), dtype='float64')
print('Means:%s, Stds:%s' %(means, stds))
#per-channel standardization of pixels
pixels = (pixels - means) / stds
#confimr it had the desired effect
means = pixels.mean(axis=(0,1), dtype='float64')
stds = pixels.std(axis=(0,1), dtype='float64')
print('Means:%s, Stds:%s' % (means, stds))


Means:[158.43480487 159.58662109 164.9829202  255.        ], Stds:[70.63586854 70.73750037 70.1171148   0.        ]
Means:[-3.98300453e-13 -1.93157989e-13  3.25967320e-13             nan], Stds:[ 1.  1.  1. nan]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
